<h1> Detecting Whether the Inputs are a Sheik </h1>

We want to train a binary classifier to accurately predict whether a multi-channel time series (representing a Super Smash Bros. Melee player's inputs) was produced by a Sheik player. We first load our required libraries.

In [ ]:
import os as os
import numpy as np
import pandas as pd

import slippi as slp

<h2> Preliminary Functions </h2>

We use these functions to one-hot encode the button bitmask and get the frame data for a given port number and frames object.

In [ ]:
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

def get_frame_data(frames, port):
    sheik_inputs = []
    for i, frame in enumerate(frames):
        buttons =  one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        buttons.extend([j_x, j_y, c_x, c_y, t_l, t_r])
        sheik_inputs.append(buttons)

    return sheik_inputs

<h2> Data Loading </h2>

We begin by iterating through the Slippi Public Dataset, extracting replays of Sheik-Fox games:

In [ ]:
dataset_path = './Slippi_Public_Dataset_v3/'

# List of all files in the dataset
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp')]

# Load the first three .slp files
for i, slp_file in enumerate(slp_files):
    
    # Get file path and store game variable
    file_path = os.path.join(dataset_path, slp_file)
    game = slp.Game(file_path)
    frames = game.frames

    # List occupied ports
    occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = occupied_ports[0]
    port_2 = occupied_ports[1]

    # Determine characters playing
    port_1_character = game.start.players[port_1].character.name
    port_2_character = game.start.players[port_2].character.name

    # Skip replay if not Sheik-Fox
    if port_1_character == 'SHEIK':
        df_row = [get_frame_data(frames, port_1), 1]
        
    elif port_2_character == 'SHEIK':
        df_row = [get_frame_data(frames, port_2), 1]
    else:
        df_row = [get_frame_data(frames, port_1), 0]
        df_row = [get_frame_data(frames, port_2), 0]
